In [201]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import os,glob
from tqdm import tqdm

In [202]:
dir_root = "/home/data/kbh/DCASE_eval/"
dir_dev  = "/home/data/kbh/DCASE_eval/dev_label/"
dir_eval = ""
n_track = 3
n_class = 13

In [203]:
def optimal_eval(path_pred,path_target):
    target = pd.read_csv(
    path_target,
    names=["idx","1","2","3","4","5"],
    index_col="idx",
    #dtype=np.int32,
    keep_default_na=False
    )
    
    pred = np.load(path_pred)
    
    display(target.head)
    

In [219]:
def optimal_dev(path_pred,path_target,list_thr):
    t = pd.read_csv(
    path_target,
    names=["idx","class","order","azim","elev"],
    #dtype=np.int32,
    keep_default_na=False
    )
    p = np.load(path_pred)
    
    
    # [n_track, n_class,n_thrshold]
    n_frame = p.shape[0]
    n_thr = len(list_thr)
    
    thr = np.zeros((n_track,n_class,n_thr))
    for i in range(n_thr) : 
        thr[:,:,i] = list_thr[i]
    TP  = np.zeros((n_track,n_class,n_thr))
    FP  = np.zeros((n_track,n_class,n_thr))
    FN  = np.zeros((n_track,n_class,n_thr))
    TN  = np.zeros((n_track,n_class,n_thr))
    
    # merge label
    label = np.zeros((n_frame,n_class),np.int16)
    label[:,:] = -1
    for i in range(len(t.index)) :
        idx_t = t.iloc[i][0]
        idx_c = t.iloc[i][1]
        
        if idx_t >= n_frame : 
            print("{} | idx_t {} >= n_frame{}".format(path_pred,idx_t,n_frame))
            break
        label[idx_t,idx_c]=int(1)
        
    # evaluate threshold
    
    for it_t in range(n_frame):
        for it_track in range(n_track) : 
            for it_c in range(n_class) :
                for it_thr in range(n_thr) : 
                    
                    # pred positive
                    if p[it_t, it_track, it_c] > thr[it_track, it_c, it_thr] : 
                        # true
                        if label[it_t,it_c] == 1:
                            TP[it_track,it_c,it_thr] +=1
                        # false
                        else :
                            FP[it_track,it_c,it_thr] +=1
                    # pred negative
                    else : 
                        # true
                        if label[it_t,it_c] == 1:
                            FN[it_track,it_c,it_thr] +=1
                        # false
                        else :
                            TN[it_track,it_c,it_thr] +=1

    
    return TP,FP,FN,TN

In [227]:
dir_pred = "/home/data/kbh/DCASE_eval/615_val/"
list_pred = [x for x in glob.glob(os.path.join(dir_pred,"*.npy"))]

In [228]:
list_thr = [0.0001, 0.001,0.005,0.01,0.05, 0.1,0.2,0.3,0.5,0.7,0.8,0.9,0.95,0.99]
n_thr = len(list_thr)
TP  = np.zeros((n_track,n_class,n_thr))
FP  = np.zeros((n_track,n_class,n_thr))
FN  = np.zeros((n_track,n_class,n_thr))
TN  = np.zeros((n_track,n_class,n_thr))
for path_pred in tqdm(list_pred) :
    name_pred = path_pred.split('/')[-1]
    part_pred = name_pred.split('_')
    path_target = os.path.join(dir_dev,part_pred[0]+"_"+part_pred[1]+"_"+part_pred[2]+".csv")
    t_TP,t_FP,t_FN,t_TN = optimal_dev(path_pred,path_target,list_thr)
    TP += t_TP
    FP += t_FP
    FN += t_FN
    TN += t_TN
recall = TP/(TP+FN)
f1 = (2*TP)/(2*TP+FP+FN)
acc = (TP+TN)/(TP+TN+FP+FN)

print(f1.shape)
for it_class in range(n_class) :
    for it_track in range(n_track):
        print("class {} | track {} | f1 ".format(it_class, it_track), end = "")
        
        # print best
        idx_max = np.argmax(f1[it_track,it_class,:])
        val_max = f1[it_track,it_class,idx_max]
        thr_max = list_thr[idx_max]
        print("                      max thr {} : {}".format(thr_max,val_max))


print(acc.shape)
for it_class in range(n_class) :
    for it_track in range(n_track):
        print("class {} | track {} | acc ".format(it_class, it_track), end = "") 
        
        # print best
        idx_max = np.argmax(acc[it_track,it_class,:])
        val_max = acc[it_track,it_class,idx_max]
        thr_max = list_thr[idx_max]
        print("                      max thr {} : {}".format(thr_max,val_max))

 72%|█████████████████████████████████▉             | 39/54 [01:27<00:27,  1.81s/it]

/home/data/kbh/DCASE_eval/615_val/fold4_room23_mix014_pred_sed.npy | idx_t 1200 >= n_frame1200


100%|███████████████████████████████████████████████| 54/54 [01:58<00:00,  2.20s/it]

(3, 13, 14)
class 0 | track 0 | f1                       max thr 0.0001 : 0.46610150021825536
class 0 | track 1 | f1                       max thr 0.001 : 0.5445580907421977
class 0 | track 2 | f1                       max thr 0.005 : 0.530916700418523
class 1 | track 0 | f1                       max thr 0.0001 : 0.6073305867310698
class 1 | track 1 | f1                       max thr 0.001 : 0.7054597123362958
class 1 | track 2 | f1                       max thr 0.001 : 0.7092018660239877
class 2 | track 0 | f1                       max thr 0.01 : 0.5617021276595745
class 2 | track 1 | f1                       max thr 0.9 : 0.48861911987860396
class 2 | track 2 | f1                       max thr 0.3 : 0.5207547169811321
class 3 | track 0 | f1                       max thr 0.001 : 0.05381875741986545
class 3 | track 1 | f1                       max thr 0.01 : 0.22481625594466062
class 3 | track 2 | f1                       max thr 0.1 : 0.42898134863701576
class 4 | track 0 | f1        

In [ ]:
f1,acc,recall = optimal_dev(path_pred,path_target)

In [ ]:
                            
    recall = TP/(TP+FN)
    f1 = (2*TP)/(2*TP+FP+FN)
    acc = (TP+TN)/(TP+TN+FP+FN)
    
    recall = np.nan_to_num(recall, nan=-1)
    f1 = np.nan_to_num(f1, nan=-1)
    acc = np.nan_to_num(acc, nan=-1)

(3, 13, 9)
class 0 | track 0 | f1  -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 
class 0 | track 1 | f1  -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 
class 0 | track 2 | f1  0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 

class 1 | track 0 | f1  0.374 0.201 0.167 0.117 0.080 0.057 0.034 0.027 0.021 
class 1 | track 1 | f1  0.038 0.024 0.014 0.010 0.000 0.000 0.000 0.000 0.000 
class 1 | track 2 | f1  0.888 0.878 0.868 0.861 0.858 0.856 0.848 0.847 0.845 

class 2 | track 0 | f1  0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 
class 2 | track 1 | f1  0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 0.000 
class 2 | track 2 | f1  0.683 0.684 0.722 0.722 0.722 0.812 0.812 0.839 0.839 

class 3 | track 0 | f1  -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 
class 3 | track 1 | f1  0.000 0.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 
class 3 | track 2 | f1  0.000 0.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.000 -1.00